In [1]:
import pickle
import copy
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import networkx as nx

In [2]:
fp = open('./data/twitch_missing_edges_final.p', "rb")
missing_edges = pickle.load(fp)
type(missing_edges)

set

In [3]:
df_neg = pd.DataFrame(list(missing_edges), columns=['Source', 'Destination'])
print df_neg.shape
df_neg.head(1)

(35324, 2)


,Source,Destination
0,12731,58618


In [4]:
df_pos = pd.read_csv('./data/musae_ENGB_edges.csv')
df_pos = df_pos.rename(columns = {'from':'Source', 'to':'Destination'})
df_pos = df_pos.drop_duplicates()
print df_pos.shape
df_pos.head(1)

(35324, 2)


,Source,Destination
0,6194,255


In [5]:
df_pos['Class'] = 1
df_pos.head(5)

,Source,Destination,Class
0,6194,255,1
1,6194,980,1
2,6194,2992,1
3,6194,2507,1
4,6194,986,1


In [6]:
df_neg['Class'] = 0
df_neg.head(5)

,Source,Destination,Class
0,12731,58618,0
1,32833,7918,0
2,48218,44197,0
3,39577,18766,0
4,5499,11590,0


In [7]:
frames = [df_pos, df_neg]
df = pd.concat(frames)
print(df.shape)
df.head(5)

(70648, 3)


,Source,Destination,Class
0,6194,255,1
1,6194,980,1
2,6194,2992,1
3,6194,2507,1
4,6194,986,1


In [8]:
df.to_csv('./data/twitch_subset.csv')

In [9]:
df_copy = copy.deepcopy(df)

## Feature Extraction

In [10]:
g = nx.from_pandas_edgelist(df[['Source','Destination']], source='Source', target='Destination',create_using=nx.DiGraph())
print nx.info(g)

Name: 
Type: DiGraph
Number of nodes: 50107
Number of edges: 70648
Average in degree:   1.4099
Average out degree:   1.4099


In [11]:
# Page Rank
pr = nx.pagerank(g)
df['Page_Rank_Src'] = df.Source.apply(lambda row: pr.get(row))
df['Page_Rank_Dst'] = df.Destination.apply(lambda row: pr.get(row))

In [12]:
# Shortest Path
def get_shortest_path(x, y):
    d = -1
    try:
        if g.has_edge(x, y):
            g.remove_edge(x, y)
            d = nx.shortest_path_length(g, source=x, target=y)
            g.add_edge(x, y)
        else:
            d = nx.shortest_path_length(g, source=x, target=y)
    except:
        d = -1
    return d

df['Shortest_Path'] = df.apply(lambda row: get_shortest_path(row['Source'], row['Destination']), axis = 1)

In [13]:
# Follows Back
def get_follows_back(x, y):
    return 1 if g.has_edge(y, x) else 0

df['Follows_Back'] = df.apply(lambda row: get_follows_back(row['Source'], row['Destination']), axis = 1)

In [14]:
# Follow Features
followers_src, followers_dst, followees_src, followees_dst, int_followers, int_followees = [], [], [], [], [], []

for i, r in df.iterrows():
    pre_src = set(g.predecessors(r['Source'])) if set(g.predecessors(r['Source'])) else set()
    suc_src = set(g.successors(r['Source'])) if set(g.successors(r['Source'])) else set()

    pre_dst = set(g.predecessors(r['Destination'])) if set(g.predecessors(r['Destination'])) else set()
    suc_dst = set(g.successors(r['Destination'])) if set(g.successors(r['Destination'])) else set()

    followers_src.append(len(pre_src))
    followees_src.append(len(suc_src))

    followers_dst.append(len(pre_dst))
    followees_dst.append(len(suc_dst))

    int_followers.append(len(pre_src.intersection(pre_dst)))
    int_followees.append(len(suc_src.intersection(suc_dst)))
        
df['Followers_Src'] = followers_src
df['Followees_Src'] = followees_src
df['Followers_Dst'] = followers_dst
df['Followees_Dst'] = followees_dst
df['Int_Followers'] = int_followers
df['Int_Followees'] = int_followees

In [15]:
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Follows_Back,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,6194,255,1,0.000033,0.000015,-1,0,0,4,0,3,0,0
1,6194,980,1,0.000033,0.000016,-1,0,0,4,0,15,0,1
2,6194,2992,1,0.000033,0.000017,4,0,0,4,3,0,0,0
3,6194,2507,1,0.000033,0.000043,3,0,0,4,8,13,0,0
4,6194,986,1,0.000033,0.000020,8,0,0,4,1,18,0,0


In [16]:
df.to_csv('./data/twitch_final_dataset.csv')